<a href="https://colab.research.google.com/github/raynardj/python4ml/blob/master/experiments/rl/atari_pooyan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pooyan with DQN

In [0]:
%matplotlib inline

Install modules

In [2]:
!apt-get install -y xvfb

!pip install pyvirtualdisplay
!pip install piglet

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

!pip install gym
!pip install gym[atari]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


Import libraries and modules

In [0]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image


import torch
from torch import nn,optim
from torch.nn import functional as F
from torchvision import transforms as T

## Set up environment

In [6]:
env = gym.make("Pooyan-v0").unwrapped

# set up matplotlib
is_ipy = 'inline' in matplotlib.get_backend()
if is_ipy:
    from IPython import display
    
plt.ion()

# if GPU is used
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("We are using IPython", is_ipy)
print("We are on device", dev)

We are using IPython True
We are on device cuda


## Replay Memory

In [0]:
Transition = namedtuple("Transition",
                        ("state","action","next_state","reward"))

class Replay_Memory(object):
    def __init__(self,capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0
        
    def push(self,*args):
        """
        args: in order: state, action, next_state, reward
        """
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position+1) % self.capacity
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
        
    def __len__(self):
        return len(self.memory)

## DQN Algorithm